# Section 6: Experiment Viewer（Streamlitアプリ）

## 🎯 このセクションの目標
- **Streamlit in Snowflake** で実験結果を閲覧できるアプリを作成
- 過去の実験のメトリクス、特徴量重要度、SHAP値を比較できるようにする

## 📋 なぜ必要？
- Snowsight UIでは実験結果の詳細な比較が難しい
- 過去の実験を振り返る際に、毎回ノートブックでコードを書くのは面倒
- → **ワンクリックで過去の実験を可視化**できるアプリがあると便利！

---

## 💡 完成イメージ

```
┌─────────────────────────────────────────────────────────┐
│  🔬 Experiment Viewer                                   │
├─────────────────────────────────────────────────────────┤
│  📊 比較ビュー | 🔍 詳細ビュー                          │
├─────────────────────────────────────────────────────────┤
│                                                         │
│  ┌──────────────────────────────────────────────────┐  │
│  │ Run      | Test F1 | Train F1 | Gap  | ROC-AUC  │  │
│  │ Baseline | 0.7800  | 0.8100   | 0.03 | 0.8500   │  │
│  │ DeepTree | 0.7650  | 0.9200   | 0.15 | 0.8300   │  │
│  │ Conserv  | 0.7900  | 0.8000   | 0.01 | 0.8600   │  │
│  └──────────────────────────────────────────────────┘  │
│                                                         │
│  📈 メトリクス比較（棒グラフ）                          │
│  📈 特徴量重要度比較（棒グラフ）                        │
└─────────────────────────────────────────────────────────┘
```


---
## 6.1 Streamlitアプリの作成

### 🚀 SQLで自動作成（推奨）

Snowsightで以下のSQLを実行するだけで、Streamlitアプリが自動作成されます。

**方法1: setup_after_04.sql を実行**
```sql
-- setup/setup_after_04.sql を Snowsight のワークシートで実行
```

**方法2: 以下を直接実行**
```sql
USE ROLE ACCOUNTADMIN;
USE DATABASE MLOPS_HOL_DB;
USE SCHEMA FEATURE_STORE;

-- Gitリポジトリを最新に同期
ALTER GIT REPOSITORY MLOPS_HOL_DB.PUBLIC.MLOPS_HOL_REPO_KE FETCH;

-- Experiment Viewer アプリを作成
CREATE OR REPLACE STREAMLIT MLOPS_HOL_DB.FEATURE_STORE.EXPERIMENT_VIEWER
  FROM '@MLOPS_HOL_DB.PUBLIC.MLOPS_HOL_REPO_KE/branches/main/streamlit/'
  MAIN_FILE = 'experiment_viewer.py'
  QUERY_WAREHOUSE = MLOPS_HOL_SQL_WH
  COMMENT = 'MLOps Hands-on - Experiment Viewer';

-- 権限付与
GRANT USAGE ON STREAMLIT MLOPS_HOL_DB.FEATURE_STORE.EXPERIMENT_VIEWER TO ROLE MLOPS_HOL_ROLE;

SELECT '🎉 Experiment Viewer 作成完了！' AS STATUS;
```

> 💡 **Git統合**を使っているので、コードを手動で貼り付ける必要はありません！


---
## 6.2 アプリを開く

### 手順

1. Snowsight左メニュー → **Projects** → **Streamlit**
2. **EXPERIMENT_VIEWER** をクリック
3. アプリが起動します！

### 📊 比較ビュー

| 機能 | 説明 |
|------|------|
| メトリクス比較表 | 全RunのTest F1, Train F1, Gap, ROC-AUC, Accuracy |
| 過学習検出 | Gap（Train F1 - Test F1）で過学習を検出 |
| 棒グラフ比較 | メトリクスと特徴量重要度を視覚的に比較 |

### 🔍 詳細ビュー

| 機能 | 説明 |
|------|------|
| メトリクス表示 | F1, ROC-AUC, Accuracy, Precision, Recall |
| パラメータ表示 | 各Runのハイパーパラメータ |
| 特徴量重要度 | 棒グラフで重要度を可視化 |
| SHAP値 | 平均絶対SHAP値を棒グラフで表示 |


---
## 6.3 アプリの構成

### ファイル構成

```
mlops_snowflake_handson/
└── streamlit/
    └── experiment_viewer.py   # Streamlitアプリのコード
```

### 主要なコード

```python
import streamlit as st
from snowflake.snowpark.context import get_active_session
import altair as alt

# セッション取得
session = get_active_session()

# 実験結果テーブルを読み込み
df = session.table("MLOPS_HOL_DB.FEATURE_STORE.EXPERIMENT_RESULTS").to_pandas()

# 比較テーブル表示
st.dataframe(df[["RUN_NAME", "F1_SCORE", "TRAIN_F1_SCORE", "OVERFIT_GAP_F1"]])

# Altairで棒グラフ
chart = alt.Chart(metrics_df).mark_bar().encode(
    x="Run:N",
    y="Value:Q",
    color="Run:N"
)
st.altair_chart(chart)
```

### 技術ポイント

| ポイント | 説明 |
|---------|------|
| `get_active_session()` | Streamlit内でSnowparkセッションを取得 |
| `session.table().to_pandas()` | Snowflakeテーブルをpandas DataFrameに変換 |
| `alt.Chart()` | Altairで動的なグラフを生成 |
| `st.tabs()` | タブで画面を切り替え |


---
## ✅ Section 6 完了！

### 学んだこと
- **Streamlit in Snowflake** でアプリを作成
- **Git統合** でStreamlitコードを管理（手動コピペ不要）
- Experiment Trackingのデータを可視化

### 作成したもの
- Streamlit App: `EXPERIMENT_VIEWER`
  - 📊 比較ビュー: 全Runのメトリクス比較、過学習検出
  - 🔍 詳細ビュー: パラメータ、特徴量重要度、SHAP値

### 💡 活用シーン
- **実験レビュー**: 過去の実験結果をチームで共有
- **モデル選定**: 複数モデルの比較検討
- **過学習検出**: Train F1とTest F1のGapを監視
- **ドキュメント**: スクリーンショットでレポート作成


---
## 🎉 ハンズオン完了！

お疲れさまでした！以下のMLOpsコンポーネントを体験しました：

| Section | 内容 | 機能 |
|---------|------|------|
| 01 | データ探索 | チャーン定義・ラベル作成 |
| 02 | Feature Store | 特徴量のバージョン管理 |
| 03 | Model Training | XGBoost、CV、SHAP |
| 04 | Experiment Tracking | 複数実験の記録・比較 |
| 05 | Model Registry | モデル登録・本番推論 |
| 06 | Experiment Viewer | Streamlitで実験可視化 |

これらを組み合わせることで、**本番運用可能なMLOpsパイプライン**を構築できます！
